In [4]:
import pandas as pd

df = pd.read_csv('Archive/summarized_results.csv')

# Sort the dataframe

df_sorted = df.drop(columns=['nn_model', 'number_of_clients'])
# Sort the dataframe by skew, dataset, and heterogeneity_class

# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
# Add a newline before each occurrence of 'fedavg' in the 'exp_type' column
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
# Sort the dataframe by the defined order
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('results.xlsx', index=False)

In [5]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/noqs.csv')
# Define the order and bold the 'cornflqs' exp_type
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('noqs.xlsx', index=False)

In [6]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/qs1.csv')
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs1.xlsx', index=False)


In [7]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/qs2.csv')
# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted = df
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs2.xlsx', index=False)

In [25]:
def algo_ranking(dataset=None, skew=None, heterogeneity_class=None):
    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv('Archive/summarized_results.csv')
    df.fillna('None', inplace=True)
    df['exp_type'] = df['exp_type'] + '_' + df['params']
    
    
    # Filter the dataframe based on the input parameters
    if dataset:
        df = df[df['dataset'] == dataset]
    if skew:
        if skew == 'None':
            df = df[df['skew'] == skew]
        elif skew == 'quantity-skew':
            df = df[df['skew'] != 'None']
        else:
            df = df[df['skew'] == skew]
    if heterogeneity_class:
        df = df[df['heterogeneity_class'] == heterogeneity_class]

    # Step 2: Clean the data
    # Remove the '\pm' and uncertainty values from the 'accuracy' column
    df['accuracy'] = df['accuracy'].str.split(' ').str[0].astype(float)
    # Replace 'n/a' with NaN for missing values
    

    # Convert metric columns to numeric
    metrics = ['accuracy', 'ARI', 'AMI', 'hom', 'cmplt', 'vm']
    df[metrics] = df[metrics].apply(pd.to_numeric, errors='coerce')

    # Concatenate 'dataset', 'heterogeneity_class', and 'skew' into one column named 'group'
    df['group'] = df['dataset'] + '_' + df['heterogeneity_class'] + '_' + df['skew']
    # Drop the original 'dataset', 'heterogeneity_class', and 'skew' columns
    df = df.drop(columns=['dataset', 'heterogeneity_class', 'skew'])

    # Step 3: Group the data by the new 'group' column
    grouped = df.groupby('group')

    # Step 4: Rank algorithms within each group for each metric
    def rank_algorithms(group):
        for metric in metrics:
            group[f'{metric}_rank'] = group[metric].rank(ascending=False, method='min')
        return group

    ranked_groups = grouped.apply(rank_algorithms)
    best = ranked_groups.groupby('exp_type')[[f'{metric}_rank' for metric in metrics]].mean()
    best = best.sort_values(by='accuracy_rank')

    return best


In [26]:
algo_ranking()

/tmp/ipykernel_59442/2146592866.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_59442/2146592866.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
hcfl,3.038462,2.307692,2.307692,2.192308,2.576923,2.269231
cornflqs,3.283019,2.037736,2.075472,1.943396,2.433962,2.075472
oracle-centralized,3.500000,NaN,NaN,NaN,NaN,NaN
ifca,3.925926,3.074074,2.962963,3.111111,1.851852,2.962963
cfl,4.851852,4.000000,4.037037,3.851852,3.703704,4.000000
fedgroup,5.961538,4.000000,4.115385,4.038462,4.846154,4.153846
fedprox,8.269231,NaN,NaN,NaN,NaN,NaN
srfca,8.592593,3.185185,3.111111,3.259259,1.000000,3.148148
fedavg,9.111111,NaN,NaN,NaN,NaN,NaN


In [32]:
algo_ranking(skew='quantity-skew-type-1')

/tmp/ipykernel_59442/2146592866.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_59442/2146592866.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
oracle-centralized,2.666667,NaN,NaN,NaN,NaN,NaN
cornflqs,3.111111,1.722222,1.722222,1.666667,2.222222,1.722222
ifca,3.222222,2.000000,2.000000,2.000000,1.333333,2.000000
cfl,4.333333,3.333333,3.444444,3.222222,3.111111,3.444444
hcfl,5.111111,4.555556,4.555556,4.444444,5.222222,4.444444
fedgroup,6.444444,6.000000,6.333333,6.111111,6.666667,6.333333
fedprox,8.555556,NaN,NaN,NaN,NaN,NaN
srfca,8.800000,3.700000,3.600000,3.900000,1.000000,3.600000
fedavg,9.777778,NaN,NaN,NaN,NaN,NaN


In [31]:
algo_ranking(skew='quantity-skew-type-2')

/tmp/ipykernel_59442/2146592866.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_59442/2146592866.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
hcfl,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
oracle-centralized,3.125000,NaN,NaN,NaN,NaN,NaN
ifca,3.666667,3.444444,3.111111,3.333333,2.555556,3.111111
cornflqs,3.764706,2.882353,3.000000,2.823529,3.470588,3.000000
fedgroup,5.250000,3.125000,3.125000,3.125000,4.375000,3.250000
cfl,7.111111,6.000000,6.000000,5.888889,5.333333,5.888889
fedprox,7.333333,NaN,NaN,NaN,NaN,NaN
fedavg,8.111111,NaN,NaN,NaN,NaN,NaN
srfca,8.875000,3.125000,3.000000,3.125000,1.000000,3.125000


In [28]:
algo_ranking(skew='quantity-skew')

/tmp/ipykernel_59442/2146592866.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_59442/2146592866.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
oracle-centralized,2.882353,NaN,NaN,NaN,NaN,NaN
cornflqs,3.428571,2.285714,2.342857,2.228571,2.828571,2.342857
ifca,3.444444,2.722222,2.555556,2.666667,1.944444,2.555556
hcfl,3.647059,2.882353,2.882353,2.823529,3.235294,2.823529
cfl,5.722222,4.666667,4.722222,4.555556,4.222222,4.666667
fedgroup,5.882353,4.647059,4.823529,4.705882,5.588235,4.882353
fedprox,7.944444,NaN,NaN,NaN,NaN,NaN
srfca,8.833333,3.444444,3.333333,3.555556,1.000000,3.388889
fedavg,8.944444,NaN,NaN,NaN,NaN,NaN


In [30]:
algo_ranking(skew='None')

/tmp/ipykernel_59442/2146592866.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('None', inplace=True)
/tmp/ipykernel_59442/2146592866.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_groups = grouped.apply(rank_algorithms)


,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank
exp_type,,,,,,
hcfl,1.888889,1.222222,1.222222,1.000000,1.333333,1.222222
cornflqs,3.000000,1.555556,1.555556,1.388889,1.666667,1.555556
cfl,3.111111,2.666667,2.666667,2.444444,2.666667,2.666667
oracle-centralized,4.666667,NaN,NaN,NaN,NaN,NaN
ifca,4.888889,3.777778,3.777778,4.000000,1.666667,3.777778
fedgroup,6.111111,2.777778,2.777778,2.777778,3.444444,2.777778
srfca,8.111111,2.666667,2.666667,2.666667,1.000000,2.666667
fedprox,9.000000,NaN,NaN,NaN,NaN,NaN
fedavg,9.444444,NaN,NaN,NaN,NaN,NaN
